In [18]:
%run -n main.py
dotenv = dict(read_dotenv('.env'))
openai.api_key = dotenv['OPENAI_TOKEN']

# sources

## alpaca

In [ ]:
!mkdir -p data/sources/alpaca
!curl -L https://github.com/yizhongw/self-instruct/raw/main/human_eval/user_oriented_instructions.jsonl \
    > data/sources/alpaca/user_oriented_instructions.jsonl

In [99]:
%run -n main.py
items = read_jsonl('data/sources/alpaca/user_oriented_instructions.jsonl')
alpaca_items = list(parse_alpaca(items))
random.sample(alpaca_items, 3)

[{'id': '7610be0f-8237-4035-96a8-5820219d619e',
  'source': 'alpaca',
  'source_id': 'user_oriented_task_224',
  'instruction': 'Convert natural language into a list of turn-by-turn directions.\n\n"Go west on I-10 until you hit the 101 then take it north. You will eventually see the Hollywood sign."'},
 {'id': 'a58bfb9a-4f0d-4330-8154-4afd17d573c6',
  'source': 'alpaca',
  'source_id': 'user_oriented_task_238',
  'instruction': 'Categorize the given product into one of Electronics, Computers, Smart Home, or Arts & Crafts departments.\n\n"Google Nest Learning Thermostat"'},
 {'id': '63ce006e-7056-46cd-842b-3615c2d8317b',
  'source': 'alpaca',
  'source_id': 'user_oriented_task_104',
  'instruction': 'List the personality traits that are required to be successful in the given job.\n\n"Social Media Marketer"'}]

## vicuna

In [ ]:
!mkdir -p data/sources/vicuna
!curl -L https://github.com/lm-sys/vicuna-blog-eval/raw/main/eval/table/question.jsonl \
    > data/sources/vicuna/question.jsonl

In [100]:
%run -n main.py
items = read_jsonl('data/sources/vicuna/question.jsonl')
vicuna_items = list(parse_vicuna(items))
random.sample(vicuna_items, 3)

[{'id': '2aea7930-c299-4be5-aad4-048d669c231d',
  'source': 'vicuna',
  'source_id': 53,
  'category': 'counterfactual',
  'instruction': 'What if the Black Death had not occurred in the 14th century?'},
 {'id': '5b49aa61-1300-495a-ac10-1d88a3186939',
  'source': 'vicuna',
  'source_id': 73,
  'category': 'writing',
  'instruction': 'Use an appropriate format to structure a formal letter of recommendation for a student applying to a prestigious graduate program in computer science.'},
 {'id': '7c08ea72-8c64-4b93-9bde-445f20dbe3b1',
  'source': 'vicuna',
  'source_id': 57,
  'category': 'counterfactual',
  'instruction': 'What if the Suez Canal had never been constructed?'}]

## arena

In [ ]:
!mkdir -p data/sources/arena
!curl -L curl -L https://huggingface.co/datasets/lmsys/chatbot_arena_conversations/resolve/main/data/train-00000-of-00001-cced8514c7ed782a.parquet \
    > data/sources/arena/train-00000-of-00001-cced8514c7ed782a.parquet

In [115]:
%run -n main.py
records = pd.read_parquet('data/sources/arena/train-00000-of-00001-cced8514c7ed782a.parquet').itertuples()
arena_items = list(parse_arena(records))
random.sample(arena_items, 3)

[{'id': 'd843ac4a-4b59-4b4f-bc3e-d6de101a7953',
  'source': 'arena',
  'source_id': '924c59203b4e4aeea5e627011da0b283',
  'lang': 'English',
  'instruction': 'what are hybrid text classification methods?'},
 {'id': '584f5fa3-37cc-44b4-bf28-dc2dd7e71682',
  'source': 'arena',
  'source_id': '37c2c4edb3b14e49b5920613d9b6befe',
  'lang': 'English',
  'instruction': 'What strains of cannabis have been reported by individuals with borderline personality disorder as preferable or effective in the management of their symptoms?'},
 {'id': 'bea19674-14cd-4996-a350-d68d2d0c0317',
  'source': 'arena',
  'source_id': '2005e97ef88246468f890495101550ea',
  'lang': 'English',
  'instruction': 'How big is the Eiffeltower?'}]

# orig

In [116]:
!mkdir -p data/orig

In [117]:
write_jsonl('data/orig/alpaca.jsonl', alpaca_items)
write_jsonl('data/orig/vicuna.jsonl', vicuna_items)

In [122]:
instruction_items = {
    _['instruction']: _
    for _ in arena_items
    if _['lang'] == 'English'
}
items = random.sample(list(instruction_items.values()), 1000)
write_jsonl('data/orig/arena.jsonl', items)

In [124]:
alpaca_items = list(read_jsonl('data/orig/alpaca.jsonl'))
vicuna_items = list(read_jsonl('data/orig/vicuna.jsonl'))
arena_items = list(read_jsonl('data/orig/arena.jsonl'))

# translate

In [178]:
translate_items = [
    {
        'id': _['id'],
        'instruction': _['instruction'],
        'answer': None
    }
    for _ in vicuna_items
]

In [175]:
%run -n main.py
items = [_ for _ in translate_items if not _['answer']]
queue = iter(tqdm(items))
workers = [openai_translate_worker(queue) for _ in range(10)]
await asyncio.gather(*workers);

100%|██████████| 156/156 [01:07<00:00,  2.30it/s]


In [188]:
!mkdir -p data/translate
write_jsonl('data/translate/vicuna.jsonl', translate_items)

# label studio

In [19]:
%run -n main.py
label_studio = label_studio_sdk.Client('http://localhost:8080', dotenv['LABELSTUDIO_TOKEN'])
label_studio.check_connection()

{'status': 'UP'}

In [20]:
title_projects = {
    _.title: _
    for _ in label_studio.list_projects()
}
translate_project = title_projects['translate']
classify_project = title_projects['classify']

# translate annot

In [185]:
translate_items = read_jsonl('data/translate/alpaca.jsonl')
annot_items = [translate_annot_item(_) for _ in translate_items]
random.choice(annot_items)

{'data': {'id': '2f6ac785-de76-4cb3-8d97-ad4a43aa1a22',
  'instruction': 'You need to write an email to negotiate your salary.'},
 'predictions': [{'result': [{'from_name': 'answer',
     'to_name': 'instruction',
     'type': 'textarea',
     'value': {'text': ['Напиши письмо, попроси прибавку к зарплате.']}}]}]}

In [21]:
translate_project.delete_all_tasks();
translate_project.import_tasks(annot_items);

In [27]:
annot_items = translate_project.export_tasks()
translate_items = [annot_translate_item(_) for _ in annot_items]
random.sample(translate_items, 3)

[{'id': '20496e66-cfbe-4237-b2f0-0f5e93e5588a',
  'instruction': 'What is the distance in km between Sydney and Lagos?',
  'answer': 'Каково расстояние в километрах между Сиднеем и Лагосом?'},
 {'id': '5ab615f9-0183-487d-aad2-064d02fbf6c3',
  'instruction': 'What is the average Ph of a soft drink:',
  'answer': 'Каково среднее значение pH газировки?'},
 {'id': '275458f1-13bd-41aa-a980-75eea737a26e',
  'instruction': 'Are capable of converting Rust code into python script ?',
  'answer': 'Ты умеешь преобразовывать код на Rust в скрипт на Python?'}]

In [ ]:
write_jsonl('data/translate/alpaca.jsonl', translate_items)

# classify

In [214]:
%run -n main.py
classify_items = list(read_jsonl('data/classify/items.jsonl'))
id_embeddings = read_pickle('data/classify/embeddings.pkl')

In [215]:
%run -n main.py
items = [
    _ for _ in classify_items
    if _['id'] not in id_embeddings
]
for index in tqdm(range(0, len(items), 64)):
    batch = items[index:index + 64]
    texts = [_['instruction'] for _ in batch]
    embeddings = openai_embed_batch(texts)
    for item, embedding in zip(batch, embeddings):
        id_embeddings[item['id']] = np.array(embedding)

write_pickle('data/classify/embeddings.pkl', id_embeddings)

0it [00:00, ?it/s]


In [216]:
target_items = [
    _ for _ in classify_items
    if _['tags'] and 'bad instruction' not in _['tags']
]
items = [_ for _ in classify_items if not _['tags']]

for item in tqdm(items):
    max_sim = 0
    for target_item in target_items:
        sim = cosine_sim(
            id_embeddings[item['id']],
            id_embeddings[target_item['id']]
        )
        if sim > max_sim:
            max_sim = sim
            item['tags'] = target_item['tags']
    item['max_sim'] = max_sim

100%|██████████| 394/394 [00:00<00:00, 424.85it/s]


In [218]:
ids = {_['id'] for _ in read_jsonl('data/translate/alpaca.jsonl')}


In [220]:
items = []
for item in classify_items:
    if not item.get('max_sim'):
        continue
        
#     if 'enumerate' not in item['tags']:
#         continue
        
    if item['id'] not in ids:
        continue

    items.append(item)

# items = sorted(items, key=lambda _: _['max_sim'], reverse=False)
annot_items = [classify_annot_item(_) for _ in items]
len(annot_items)

131

In [221]:
classify_project.delete_all_tasks();
classify_project.import_tasks(annot_items);

In [222]:
%run -n main.py
annot_items = classify_project.export_tasks()
items = (annot_classify_item(_) for _ in annot_items)
id_tags = {
    _['id']: _['tags']
    for _ in items
}
for item in classify_items:
    tags = id_tags.get(item['id'])
    if tags is not None:
        item['tags'] = tags
        item.pop('max_sim', None)

In [223]:
!mkdir -p data/classify
for item in classify_items:
    if item.pop('max_sim', None):
        item['tags'] = []

write_jsonl('data/classify/items.jsonl', classify_items)

# tasks

In [210]:
items = read_jsonl('data/classify/items.jsonl')
id_tags = {_['id']: _['tags'] for _ in items if _['tags']}
len(id_tags)

474

In [211]:
!mkdir -p data/tasks

for name in ['alpaca', 'vicuna', 'arena']:
    task_items = []
    items = read_jsonl(f'data/translate/{name}.jsonl')
    for item in items:
        id = item['id']
        tags = id_tags.get(id, [])
        if 'bad instruction' in tags:
            continue

        task_items.append({
            'id': id,
            'instruction': item['answer'],
            'tags': tags
        })
    write_jsonl(f'data/tasks/{name}.jsonl', task_items)

In [202]:
from contextlib import redirect_stdout
from html import escape

with open('data/tasks/README.md', 'w') as file:
    with redirect_stdout(file):
        for name in ['alpaca', 'vicuna', 'arena']:
            print(f'<h1>{name}</h1>')

            items = read_jsonl(f'data/tasks/{name}.jsonl')
            items = [_ for _ in items if _['tags']]

            for item in random.sample(items, 30):
                print(' '.join(f'<code>#{_}</code>' for _ in item['tags']))
                print('<br/>')
                instruction = escape(item['instruction'])
                print('<br/>\n'.join(instruction.splitlines()))
                print('<br/><br/>')